In [26]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
bank = pd.read_csv("https://raw.githubusercontent.com/ltdaovn/dataset/master/bank/bank-full.csv",sep = ";", na_values = "unknown")

### a. Tập dữ liệu có bao nhiêu hàng, bao nhiêu cột?

In [27]:
# Lấy số hàng và số cột
num_rows, num_columns = bank.shape

# In số hàng và số cột
print("Số hàng trong tập dữ liệu:", num_rows)
print("Số cột trong tập dữ liệu:", num_columns)

Số hàng trong tập dữ liệu: 45211
Số cột trong tập dữ liệu: 17


### b. Tập dữ liệu có bao nhiêu khách hàng? Trong đó bao nhiêu khách hàng đăng ký dịch vụ, bao nhiêu khách hàng không đăng ký. Học viên có kết luận gì về tính cân bằng của tập dữ liệu này.

In [28]:
# Sử dụng value_counts() để đếm số lượng giá trị trong cột "y"
customer_counts = bank["y"].value_counts()

# Lấy số lượng khách hàng đăng ký và không đăng ký
customers_subscribed = customer_counts["yes"]
customers_not_subscribed = customer_counts["no"]

# In số lượng khách hàng và khách hàng đăng ký, không đăng ký
print("Số lượng khách hàng đăng ký dịch vụ:", customers_subscribed)
print("Số lượng khách hàng không đăng ký dịch vụ:", customers_not_subscribed)


Số lượng khách hàng đăng ký dịch vụ: 5289
Số lượng khách hàng không đăng ký dịch vụ: 39922


### c. Cột nào có nhiều dữ liệu rỗng (null). Hãy cho biết số lượng dữ liệu rỗng của các cột này. Để xóa bỏ các record có dữ liệu rỗng, chúng ta có thể sử dụng hàm gì trong thư viện pandas.

In [29]:
# Tạo DataFrame chứa các giá trị boolean, True cho ô trống và False cho ô không trống
null_values = bank.isnull()

# Đếm số lượng dữ liệu rỗng (null) trong từng cột
null_counts = null_values.sum()

# Tìm cột có nhiều dữ liệu rỗng nhất
column_with_most_null = null_counts.idxmax()

# Số lượng dữ liệu rỗng trong từng cột
print("Số lượng dữ liệu rỗng (null) trong từng cột:")
print(null_counts)

# Tên cột có nhiều dữ liệu rỗng nhất
print("Cột có nhiều dữ liệu rỗng nhất:", column_with_most_null)


Số lượng dữ liệu rỗng (null) trong từng cột:
age              0
job            288
marital          0
education     1857
default          0
balance          0
housing          0
loan             0
contact      13020
day              0
month            0
duration         0
campaign         0
pdays            0
previous         0
poutcome     36959
y                0
dtype: int64
Cột có nhiều dữ liệu rỗng nhất: poutcome


In [30]:
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,NaN,5,may,261,1,-1,0,NaN,no
1,44,technician,single,secondary,no,29,yes,no,NaN,5,may,151,1,-1,0,NaN,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,NaN,5,may,76,1,-1,0,NaN,no
3,47,blue-collar,married,NaN,no,1506,yes,no,NaN,5,may,92,1,-1,0,NaN,no
4,33,NaN,single,NaN,no,1,no,no,NaN,5,may,198,1,-1,0,NaN,no


In [31]:
bank.shape
bank.columns

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'y'],
      dtype='object')

In [32]:
bank["default"] = bank["default"].map({"no":0,"yes":1})
bank["housing"] = bank["housing"].map({"no":0,"yes":1})
bank["loan"] = bank["loan"].map({"no":0,"yes":1})
bank["y"] = bank["y"].map({"no":0,"yes":1})
bank.education = bank.education.map({"primary": 0, "secondary":1,
"tertiary":2})
bank.month = pd.to_datetime(bank.month, format = "%b").dt.month

In [33]:
bank.isnull().sum()

age              0
job            288
marital          0
education     1857
default          0
balance          0
housing          0
loan             0
contact      13020
day              0
month            0
duration         0
campaign         0
pdays            0
previous         0
poutcome     36959
y                0
dtype: int64

In [34]:
bank.drop(["poutcome", "contact"], axis = 1, inplace = True)
bank.dropna(inplace = True)
bank = pd.get_dummies(bank, drop_first = True)

In [35]:
bank.y.value_counts()

0    38172
1     5021
Name: y, dtype: int64

In [36]:
X = bank.drop("y", axis = 1)
y = bank.y
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1,
stratify=y)
y_train.value_counts()

0    28628
1     3766
Name: y, dtype: int64

### e. Hãy cho biết số lượng khách hàng đăng ký (subscribed) và không đăng ký (not subscribed) sau khi sử dụng các giải thuật cân bằng dữ liệu

In [38]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

# X = bank.drop("y", axis = 1)
# y = bank.y
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1,
# stratify=y)

# Sử dụng SMOTE để cân bằng dữ liệu
smt = SMOTE()
X_train_smt, y_train_smt = smt.fit_resample(X_train, y_train)

# Sử dụng NearMiss để cân bằng dữ liệu
nr = NearMiss()
X_train_nr, y_train_nr = nr.fit_resample(X_train, y_train)

# Sử dụng value_counts() để đếm số lượng khách hàng đăng ký và không đăng ký trước và sau khi cân bằng
subscribed_counts_before_balance = y.value_counts()
subscribed_counts_after_balance_smt = pd.Series(y_train_smt).value_counts()
subscribed_counts_after_balance_nr = pd.Series(y_train_nr).value_counts()

# Lấy số lượng khách hàng đăng ký và không đăng ký trước và sau khi cân bằng
subscribed_customers_before_balance = subscribed_counts_before_balance[1]  # 1 đại diện cho đăng ký
not_subscribed_customers_before_balance = subscribed_counts_before_balance[0]  # 0 đại diện cho không đăng ký
subscribed_customers_after_balance_smt = subscribed_counts_after_balance_smt[1]  # 1 đại diện cho đăng ký
not_subscribed_customers_after_balance_smt = subscribed_counts_after_balance_smt[0]  # 0 đại diện cho không đăng ký
subscribed_customers_after_balance_nr = subscribed_counts_after_balance_nr[1]  # 1 đại diện cho đăng ký
not_subscribed_customers_after_balance_nr = subscribed_counts_after_balance_nr[0]  # 0 đại diện cho không đăng ký

# In số lượng khách hàng đăng ký và không đăng ký trước và sau khi cân bằng
print("Số lượng khách hàng đăng ký trước khi cân bằng dữ liệu:", subscribed_customers_before_balance)
print("Số lượng khách hàng không đăng ký trước khi cân bằng dữ liệu:", not_subscribed_customers_before_balance)
print("Số lượng khách hàng đăng ký sau khi sử dụng SMOTE:", subscribed_customers_after_balance_smt)
print("Số lượng khách hàng không đăng ký sau khi sử dụng SMOTE:", not_subscribed_customers_after_balance_smt)
print("Số lượng khách hàng đăng ký sau khi sử dụng NearMiss:", subscribed_customers_after_balance_nr)
print("Số lượng khách hàng không đăng ký sau khi sử dụng NearMiss:", not_subscribed_customers_after_balance_nr)

Số lượng khách hàng đăng ký trước khi cân bằng dữ liệu: 5021
Số lượng khách hàng không đăng ký trước khi cân bằng dữ liệu: 38172
Số lượng khách hàng đăng ký sau khi sử dụng SMOTE: 28628
Số lượng khách hàng không đăng ký sau khi sử dụng SMOTE: 28628
Số lượng khách hàng đăng ký sau khi sử dụng NearMiss: 3766
Số lượng khách hàng không đăng ký sau khi sử dụng NearMiss: 3766


In [44]:
from sklearn import svm
model = svm.SVC()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))

Accuracy:  0.5548661913140106
Recall:  0.8310756972111554


In [41]:
#SMOTE---------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1,
stratify=y)
smt = SMOTE()
X_train, y_train = smt.fit_resample(X_train, y_train)
np.bincount(y_train)
from sklearn import svm
model = svm.SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))

Accuracy:  0.7783128067413649
Recall:  0.7051792828685259


In [43]:
#NearMiss--------------------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1,
stratify=y)
nr = NearMiss()
X_train, y_train = nr.fit_resample(X_train, y_train)
np.bincount(y_train)
from sklearn import svm
model = svm.SVC()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)
print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Recall: ", recall_score(y_test, y_pred))

Accuracy:  0.5548661913140106
Recall:  0.8310756972111554
